In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import gc
import sys
import time
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, StratifiedKFold
import xgboost as xgb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from transformers import AutoModel, AutoTokenizer

In [ ]:
data_dir = '../input/commonlitreadabilityprize/'
train = pd.read_csv(data_dir + 'train.csv')
test = pd.read_csv(data_dir + 'test.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

target = train['target'].to_numpy()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape , test.shape

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [ ]:
#from transformers import BertTokenizer, BertModel
#tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
#tokenizer.save_pretrained('./')
tokenizer_fast=tokenizer.from_pretrained('./')

In [ ]:
train_tokens = []
for txt in train['excerpt']:
    token = tokenizer_fast.encode_plus(txt,pad_to_max_length=True,truncation=True,return_tensors='pt', max_length=812)
    train_tokens.append(token['input_ids'])

In [ ]:
test_tokens = []
for txt in test['excerpt']:
    token = tokenizer_fast.encode_plus(txt,pad_to_max_length=True,truncation=True,return_tensors='pt', max_length=812)
    test_tokens.append(token['input_ids'])

In [ ]:
len(train_tokens) , len(test_tokens)


In [ ]:
max_len=812

In [ ]:
columns_train=[str(i) for i in range(max_len)]
columns_test=[str(i) for i in range(max_len)]

In [ ]:
train_tokens=[t.numpy() for t in train_tokens]
test_tokens=[t.numpy() for t in test_tokens]

In [ ]:
train_tokens=np.array(train_tokens)
test_tokens=np.array(test_tokens)

In [ ]:
train_tokens=np.resize(train_tokens,(2834,812))
test_tokens=np.resize(test_tokens,(7,812))

In [ ]:
train_df=pd.DataFrame(train_tokens,columns=columns_train)
test_df=pd.DataFrame(test_tokens,columns=columns_test)

In [ ]:
train_df.shape , test_df.shape

In [ ]:
train_df['target']=train['target']

In [ ]:
predictors=columns_train
target=['target']

X=train_df[predictors].values
y=train_df[target].values

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, KFold, GridSearchCV
from sklearn.metrics import mean_squared_log_error,r2_score,mean_squared_error,mean_squared_log_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PowerTransformer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
pt_X=StandardScaler()
pt_y=StandardScaler()
X=pt_X.fit_transform(X)
y=pt_y.fit_transform(y)

In [ ]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=10, random_state=56, shuffle=True)
for train_index, test_index in kf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf1 =xgb.XGBRegressor(max_depth=15, learning_rate=0.001, n_estimators=789)
    model=clf1.fit(X,y)
    pred_train=model.predict(X_train)
    pred_test=model.predict(X_test)
    y_test=pt_y.inverse_transform(y_test)
    y_train=pt_y.inverse_transform(y_train)
    pred_train=pt_y.inverse_transform(np.reshape(pred_train,(-1,1)))
    pred_test=pt_y.inverse_transform(np.reshape(pred_test,(-1,1)))
    print("The R2 score of train",r2_score(y_train,pred_train))
    print("The R2 score of test",r2_score(y_test,pred_test))
    rmse_1=np.sqrt(mean_squared_error(y_train,pred_train))
    print(" RMSE Error on Train",rmse_1)
    rmse_2=np.sqrt(mean_squared_error(y_test,pred_test))
    print("RMSE Error on Test",rmse_2)
    print('*'*50)

In [ ]:
pt_test=StandardScaler()
test=test_df[predictors].values
test=pt_test.fit_transform(test)
pred=model.predict(test)

In [ ]:
pred_test=pt_y.inverse_transform(np.reshape(pred,(-1,1)))
sub=pd.DataFrame(test_df, columns=['id'])
sub['target']=pred_test
sub.to_csv('submission.csv', index=False)